In [5]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns',None)

In [6]:
df_home=pd.read_csv('/Users/aurelientarroux/Desktop/ML/Data_Challenge_QRT/Data/Train_Data/train_home_team_statistics_df.csv')
df_away=pd.read_csv('/Users/aurelientarroux/Desktop/ML/Data_Challenge_QRT/Data/Train_Data/train_away_team_statistics_df.csv')

df_home.columns='home_'+df_home.columns
df_away.columns='away_'+df_away.columns

df_result=pd.read_csv('/Users/aurelientarroux/Desktop/ML/Data_Challenge_QRT/Data/Y_train_1rknArQ.csv')

base_match=pd.merge(df_home,df_away,left_on="home_ID",right_on='away_ID',how='inner')
base=pd.merge(base_match,df_result,left_on="home_ID",right_on='ID',how='inner')
base=base.drop(['home_ID','home_LEAGUE','home_TEAM_NAME','away_ID','away_LEAGUE','away_TEAM_NAME'],axis=1)

def resultat(row):
    if row['HOME_WINS']==1:
        return 'HOME'
    elif row['AWAY_WINS']==1:
        return 'AWAY'
    else:
        return 'DRAW'

base['Resultat']=base.apply(resultat, axis=1)
base=base.drop(['HOME_WINS','AWAY_WINS','DRAW'],axis=1)

base.dropna(inplace=True)
base=base[base['Resultat'].isin(['HOME','AWAY'])]

X = base.drop(['Resultat','ID'],axis=1)
y = base['Resultat']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
#Fonction qui permet de renvoyer les individus pour lesquelles h renvoie vrai ou faux

def index_bon_mauvais(colonne,seuil,signe):
    
    df_individus_h=pd.DataFrame(index=X_train.index)
    
    
    individus_gauche=X_train[X_train[colonne]<seuil].index
    individus_droite=X_train[X_train[colonne]>seuil].index
    
    if signe==1:
        df_individus_h.loc[individus_gauche,'Resultat']='HOME'#rep1
        df_individus_h.loc[individus_droite,'Resultat']='AWAY'#rep2
    
    else:
        df_individus_h.loc[individus_gauche,'Resultat']='AWAY'#rep2
        df_individus_h.loc[individus_droite,'Resultat']='HOME'#rep1
    
    df_y_train=pd.DataFrame(y_train)
    df_TF=pd.DataFrame(df_individus_h==df_y_train)
    
    return [df_TF[df_TF['Resultat']==True].index,df_TF[df_TF['Resultat']==False].index]

In [8]:
def categ(df_poids):
    
    liste_signe=[1,2]
    liste_erreur=[]
    
    for colonne in X_train.columns[50:230]:
        
        #Je construit l'ensemble des seuil pour cette catégories
        a=list(X_train.loc[:,colonne])
        liste_distincte = list(set(a))
        liste_distincte.sort()
        liste_moy=[]
        for j in range(len(liste_distincte)-1):
            liste_moy+= [(liste_distincte[j+1]+liste_distincte[j])/2]
            
        #Je traite l'ensemble des seuils
        for seuil in liste_moy:
            
            #Je traite les deux signes
            for signe in liste_signe:
                
                #Je regarde les individus bon et mauvaix
                individus_bon, individus_mauvais = index_bon_mauvais(colonne,seuil,signe)
                
                #Je regarde l'erreur pour cette catégorie, ce seuil, et ce signe 
                erreur=0
                for index in individus_mauvais:
                    erreur+=df_poids.loc[index,'poids']
                
                #Je regarde quelle categ est à droite ou à gauche 
                if signe==1:
                    reponse_petit='HOME'#rep1
                    reponse_grand='AWAY'#rep2
                else:
                    reponse_petit='AWAY'#rep2
                    reponse_grand='HOME'#rep1

                liste_erreur+=[[individus_bon,individus_mauvais,erreur,colonne,seuil,signe,reponse_petit,reponse_grand]]
    liste_erreur.sort(key=lambda x: x[2]) 
    return liste_erreur[0]

In [9]:
liste_poids=[]
for k in range(len(X_train)):
    liste_poids+=[(1/len(X_train))]
df_poids=pd.DataFrame({'poids':liste_poids}, index=X_train.index)

liste_seuil_carac_reponse=[]
liste_perf_f=[]

for compte in range(100):
    
    categ_poids=categ(df_poids)
    
    index_bon=categ_poids[0]
    index_mauvais=categ_poids[1]
    erreur=categ_poids[2]
    
    
    #construction de l'erreur
    perf_f=(1/2)*np.log((1-erreur)/erreur)
    
    #construction des nouveaux poids
    for index in index_mauvais:
        df_poids.loc[index,'poids']=df_poids.loc[index,'poids']*np.exp(perf_f)
    for index in index_bon:
        df_poids.loc[index,'poids']=df_poids.loc[index,'poids']*np.exp(-perf_f)
    df_poids['poids']=df_poids['poids']/(df_poids['poids'].sum())
    
    #Ajoute des élements nécessaire à la distinction dans la construction de l'arbre
    liste_seuil_carac_reponse+=[[categ_poids[3:8],perf_f]]
    print(liste_seuil_carac_reponse)

    print(compte)
print(liste_seuil_carac_reponse)


[[['away_TEAM_SHOTS_ON_TARGET_season_sum', 6.5, 1, 'HOME', 'AWAY'], np.float64(0.2729661006593455)]]
0
[[['away_TEAM_SHOTS_ON_TARGET_season_sum', 6.5, 1, 'HOME', 'AWAY'], np.float64(0.2729661006593455)], [['home_TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 4.5, 2, 'AWAY', 'HOME'], np.float64(0.16771732961215868)]]
1
[[['away_TEAM_SHOTS_ON_TARGET_season_sum', 6.5, 1, 'HOME', 'AWAY'], np.float64(0.2729661006593455)], [['home_TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 4.5, 2, 'AWAY', 'HOME'], np.float64(0.16771732961215868)], [['away_TEAM_GAME_LOST_season_average', 2.5, 2, 'AWAY', 'HOME'], np.float64(0.149376784848081)]]
2
[[['away_TEAM_SHOTS_ON_TARGET_season_sum', 6.5, 1, 'HOME', 'AWAY'], np.float64(0.2729661006593455)], [['home_TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 4.5, 2, 'AWAY', 'HOME'], np.float64(0.16771732961215868)], [['away_TEAM_GAME_LOST_season_average', 2.5, 2, 'AWAY', 'HOME'], np.float64(0.149376784848081)], [['home_TEAM_PASSES_5_last_match_average', 3.5, 2, 'AWAY', 'HOME'], np.floa

In [10]:
y_pred=pd.DataFrame(index=y_test.index,columns=['Resultat'])

for index in X_test.index:
    
    compte_away=0
    compte_home=0
    
    for liste in liste_seuil_carac_reponse:
        
        coef=liste[1]
        
        seuil=liste[0][1]
        colonne=liste[0][0]
        signe=liste[0][2]
        
        reponse_petit=liste[0][3]
        
        if X_test.loc[index,colonne]<seuil:
            if reponse_petit=='AWAY':
                compte_away+=coef
            else:
                compte_home+=coef
            
        else:
            if reponse_petit=='HOME':
                compte_away+=coef
            else:
                compte_home+=coef
    
    if compte_away<compte_home:
        y_pred.loc[index,'Resultat']='HOME'
    else:
        y_pred.loc[index,'Resultat']='AWAY'

final=pd.DataFrame(y_pred==pd.DataFrame(y_test))

print(len(final[final['Resultat']==True])/len(y_test))

0.6545012165450121


In [21]:
liste_colonne=[]
for liste in liste_seuil_carac_reponse:
    liste_colonne+=[liste[0][0]]
    
compteur = pd.Series(liste_colonne).value_counts()

compteur.head(10)


away_TEAM_SHOTS_ON_TARGET_season_sum          4
away_TEAM_GAME_LOST_season_average            3
away_TEAM_SHOTS_TOTAL_season_std              3
away_TEAM_ATTACKS_5_last_match_sum            2
away_TEAM_CORNERS_season_sum                  2
home_TEAM_SHOTS_INSIDEBOX_5_last_match_sum    2
away_TEAM_GOALS_5_last_match_sum              2
away_TEAM_CORNERS_season_std                  2
away_TEAM_BALL_SAFE_season_average            2
home_TEAM_ATTACKS_season_std                  2
Name: count, dtype: int64